In [2]:
import pandas as pd
import sbol2
import os
import requests
from dotenv import load_dotenv
import zipfile
from __future__ import absolute_import, division, print_function
import numpy as np
import os
import subprocess
import sys
import tempfile
from abc import abstractmethod, ABCMeta
from sklearn.tree import DecisionTreeClassifier as scikit_DecisionTree
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
import sys
import argparse
import time
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
# import xgboost as xgb
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_columns', None)

current_dir = os.path.abspath('')
data_path = os.path.join(current_dir, '..', 'data')
attachments_path = os.path.join(current_dir, '..', 'attachments')
pulled_attachments_path = os.path.join(current_dir, '..', 'pulled_attachments')
sbol_path = os.path.join(current_dir, '..', 'sbol_data')
downloaded_sbol_path = os.path.join(current_dir, '..', 'downloaded_sbol')
original_data_path = os.path.join(data_path, 'original_data')
scripts_path = os.path.join(current_dir, 'scripts')
model_data_path = os.path.join(data_path, 'processed_data', 'replicated_models')
model_output_path = os.path.join('..', 'model_outputs')

load_dotenv()

True

In [3]:
df = pd.read_csv(os.path.join(original_data_path, "frag-rLP5_LB_expression.txt"), delimiter=" ")
# Expression levels from genomic fragment MPRA (random 200–300 bp sheared fragments) in LB media	
df2 = pd.read_csv(os.path.join(original_data_path, "frag-rLP5-M9_expression.txt"), delimiter=" ")
# Expression levels from genomic fragment MPRA (random 200–300 bp sheared fragments) in M9 media at rLP5

In [4]:
df3 = pd.read_csv(os.path.join(original_data_path, "fLP3_Endo2_lb_expression_formatted.txt"), delimiter=" ")
df3 = df3.dropna(subset=['start'])

In [5]:
df3

,name,tss_name,tss_position,strand,start,end,variant,RNA_exp_sum_1_1,RNA_exp_sum_1_2,RNA_exp_sum_2_1,RNA_exp_sum_2_2,RNA_exp_sum_1,RNA_exp_sum_2,RNA_exp_sum_ave,DNA_1,DNA_2,DNA_ave,expn_med,num_barcodes_mapped,num_barcodes_integrated,category
0,"TSS_11125_storz_regulondb,2945404,+",TSS_11125_storz_regulondb,2945404.0,+,2945284.0,2945434.0,GCAAATTTTGCACAAAAAATAGGCTTTAGTGATTTGTTTTTGTTCA...,43.807236,55.833039,54.661823,49.577995,49.820137,52.119909,50.970023,31.625906,35.101752,33.363829,52.010547,50,28,tss
1,"TSS_1218_storz,289782,+",TSS_1218_storz,289782.0,+,289662.0,289812.0,GTATCTGCCTCCGATTCTCTGCAGAAGCAGAAAGACATTGGATCGA...,0.562627,0.549348,0.816074,0.585183,0.555988,0.700629,0.628308,36.298833,31.606403,33.952618,0.565387,53,26,tss
2,"TSS_12352_storz,3253602,-",TSS_12352_storz,3253602.0,-,3253572.0,3253722.0,CGTTTGTCTGCGCTGTGTGCCGCAACGACCGCAGCAATGGGGGCCG...,0.516503,0.717569,0.658121,0.587724,0.617036,0.622923,0.619979,72.105393,72.498105,72.301749,0.577755,108,54,tss
3,"TSS_6478_storz,1684674,-",TSS_6478_storz,1684674.0,-,1684644.0,1684794.0,GCAGATACAACTCACACAATGCACCCGCTGTGTGAAATAAACAGAG...,1.006087,1.084363,0.763596,0.900281,1.045225,0.831939,0.938582,35.065895,37.408634,36.237265,0.936175,56,27,tss
4,"TSS_2956_storz,770940,+",TSS_2956_storz,770940.0,+,770820.0,770970.0,TTATAAAGATATGACCAAGTTCTGGGGCAAGTTGTTTGGTATCAAC...,0.715541,0.744280,0.679240,0.634099,0.729911,0.656670,0.693290,79.950519,78.564136,79.257327,0.632548,116,59,tss
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17533,"TSS_6128_storz,1599413,+",TSS_6128_storz,1599413.0,+,1599293.0,1599443.0,AATCTTTAATGCAATTGTTCAGTTCTTGCTCATTTATATCTGTGAT...,0.339038,0.703175,0.504604,1.300406,0.521106,0.902505,0.711806,3.589814,3.511266,3.550540,0.574114,9,3,tss
17534,"TSS_8744_storz,2304785,-",TSS_8744_storz,2304785.0,-,2304755.0,2304905.0,AAATTATAATGTTTCTAAAATTAGAATATAATTTATAAACATTATT...,1.112467,1.246183,1.027605,1.300406,1.179325,1.164006,1.171665,4.678904,4.427289,4.553096,1.234837,3,3,tss
17535,"TSS_14532_regulondb,3740743,+",TSS_14532_regulondb,3740743.0,+,3740623.0,3740773.0,ATTGATTTTTATGTCTTTTGAAATTCATCAATCAGATTGCCTTGTT...,1.695188,1.104990,0.446785,1.253963,1.400089,0.850374,1.125232,2.379770,1.978044,2.178907,1.168341,3,3,tss
17536,"TSS_5145_wanner,1306695,-",TSS_5145_wanner,1306695.0,-,1306665.0,1306815.0,TCATAGATTAGAAATCCTTTTGAAAGCGCATTGTTATGAGTTTACG...,1.331934,1.031324,1.572683,1.300406,1.181629,1.436545,1.309087,3.428542,2.855406,3.141974,1.409033,14,4,tss


In [12]:
def create_sbol_attempt2_doc(df, file_path, media="LB"):
    doc = sbol2.Document()
    sbol2.setHomespace('http://github.com/cywlol/promoters')
    doc.displayId = "E_coli_promoters"

    media_label_MD = media
    chassis_label_MD = "E_coli_chassis"
    sample_design_labels = []
    
    # define the media once, then have it as a reference for each promoter row
    media_md = sbol2.ModuleDefinition(media_label_MD)
    media_md.addRole("http://identifiers.org/ncit/NCIT:C48164")
    media_md.version = "1"
    doc.addModuleDefinition(media_md)

    # define the chassis once, then have it as a reference for each promoter row
    chassis_md = sbol2.ModuleDefinition(chassis_label_MD)
    chassis_md.addRole("http://identifiers.org/ncit/NCIT:C14419")
    chassis_md.wasDerivedFrom = ["https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=511145"]
    chassis_md.version = "1"
    doc.addModuleDefinition(chassis_md)

    for i, row in df.iterrows():
        fragment_seq = row["variant"]
        start = row["start"]
        end = row["end"]
        strand = row["strand"]
        RNA_exp_ave = row["expn_med"]

        # Establish the identities of each component, ensuring that each one is unique
        promoter_label_CD = f"promoter_definition_{i}"
        promoter_label_FC = f"promoter_fc_{i}"
        promoter_seq_label = f"promoter_seq_{i}"
        sample_design_label_MD = f"sample_design_definition_{i}"
        chassis_module_label = f"chassis_module_{i}"
        circuit_label_CD = f"circuit_definition_{i}"
        strain_label_MD = f"strain_definition_{i}"
        circuit_label_FC = f"cicuit_fc_{i}"
        measurement_label = f"exp_ave_{i}"
        interaction_label = f"interaction_{i}"
        gfp_fc_label = f"gfp_{i}"
        promoter_location_label = f"promoter_{i}_location"
        strain_module_label = f"strain_module_{i}"
        media_module_label = f"media_module_{i}"
        # define gfp once
        gfp_component_label = f"gfp_{i}"

        sample_design_labels.append(sample_design_label_MD)
        #engr_region_label_CD = f"engr_region_{i}"
        #location_promoter_annotation = f"location_promoter_annotation_{i}"
        #location_promoter_label = f"location_promoter_label_{i}"


        '''
        # Engineered Region
        engineered_cd = sbol2.ComponentDefinition(engr_region_label_CD, sbol2.BIOPAX_DNA)
        engineered_cd.roles = ["https://identifiers.org/so/SO:0000804"]
        sub = engineered_cd.components.create('promoter')
        sub.definition = promoter_cd.persistentIdentity
        doc.addComponentDefinition(engineered_cd)
        # No sequence?
        '''


        # Add strain module definition
        strain_md = sbol2.ModuleDefinition(strain_label_MD)
        strain_md.addRole("http://identifiers.org/ncit/NCIT:C14419")
        strain_md.version = "1"
        doc.addModuleDefinition(strain_md)

        chassis_module = sbol2.Module(chassis_module_label)
        chassis_module.definition = chassis_md.identity
        strain_md.modules.add(chassis_module)

        # Sample Design
        sample_md = sbol2.ModuleDefinition(sample_design_label_MD)
        sample_md.addRole("http://identifiers.org/obo/OBI:0000073")
        sample_md.version = "1"
        doc.addModuleDefinition(sample_md)

        # Add strain module to sample design
        strain_module = sbol2.Module(strain_module_label)
        strain_module.definition = strain_md.identity
        # Add media module to sample design
        media_module = sbol2.Module(media_module_label)
        media_module.definition = media_md.identity


        sample_md.modules.add(media_module)
        sample_md.modules.add(strain_module)

        # Promoter and Sequence
        promoter_cd = sbol2.ComponentDefinition(promoter_label_CD, sbol2.BIOPAX_DNA)
        promoter_cd.roles = [sbol2.SO_PROMOTER]
        promoter_cd.version = "1"
        doc.addComponentDefinition(promoter_cd)
        seq = sbol2.Sequence(promoter_seq_label, fragment_seq, sbol2.SBOL_ENCODING_IUPAC)
        seq.version = "1"

        doc.addSequence(seq)
        promoter_cd.sequences = seq

        # Circuit
        circuit = sbol2.ComponentDefinition(circuit_label_CD, sbol2.BIOPAX_DNA)
        circuit.roles = ["https://identifiers.org/so/SO:0000804"]
        circuit.version = "1"

        annotation = sbol2.SequenceAnnotation(promoter_location_label)
        annotation.version = "1"
        rng = sbol2.Range("prange", start, end)
        rng.version = "1"

        if (strand == "-"):
            rng.orientation = sbol2.SBOL_ORIENTATION_REVERSE_COMPLEMENT

        annotation.locations.add(rng)
        circuit.sequenceAnnotations.add(annotation)
        promoter_comp = circuit.components.create('promoter')
        promoter_comp.definition = promoter_cd.identity
        doc.addComponentDefinition(circuit)


        circuit_fc = sbol2.FunctionalComponent(circuit_label_FC)
        circuit_fc.version = "1"
        circuit_fc.definition = circuit.identity
        circuit_fc.direction = sbol2.SBOL_DIRECTION_OUT
        circuit_fc.access = sbol2.SBOL_ACCESS_PUBLIC
        strain_md.functionalComponents.add(circuit_fc)

        measurement3 = sbol2.Measurement(measurement_label, value=RNA_exp_ave, unit="http://www.ontology-of-units-of-measure.org/resource/om-2/RatioUnit")
        measurement3.version = "1"

        gfp_protein = sbol2.ComponentDefinition(gfp_component_label, sbol2.BIOPAX_PROTEIN)
        gfp_protein.roles=[sbol2.SO_CDS]
        gfp_protein.version = "1"
        doc.addComponentDefinition(gfp_protein)

        gfp_fc = sbol2.FunctionalComponent(gfp_fc_label)
        gfp_fc.version = "1"
        gfp_fc.definition = gfp_protein.identity
        gfp_fc.direction = sbol2.SBOL_DIRECTION_OUT
        gfp_fc.access = sbol2.SBOL_ACCESS_PUBLIC


        doc.add(measurement3)
        m = gfp_fc.measurements.add(measurement3)

        sample_md.functionalComponents.add(gfp_fc)
        # exp_attachment = sbol2.Attachment(exp_label)
        # exp_attachment.name = f'fragmentation expression at rLP5 for sample {i}'
        # exp_attachment.description = 'CSV including the gene expression of the sequence: RNA_exp1, RNA_exp2, and its average.'
        # exp_attachment.source = 'CSV_LINK_HERE' # update when added attachment to SBOL collection
        # exp_attachment.format = 'https://identifiers.org/edam/format_3752'
        # doc.addAttachment(exp_attachment)


    report = doc.validate()
    if (report == 'Valid.'):
        doc.write(file_path)
    else:
        print(report)
    return doc, chassis_label_MD, sample_design_labels


        
def partshop_attach_exp_data(synbio_username, collection_name, file_names, email, password, exp_data_labels, version=1):
    shop = sbol2.PartShop("https://synbiohub.org")
    print(shop.login(email, password))
    exp_labels = ["ExperimentalData_" + label for label in exp_data_labels]  
    for label, file_name in zip(exp_labels, file_names):
        path = os.path.join(attachments_path, file_name)
        attachment_uri = f"https://synbiohub.org/user/{synbio_username}/{collection_name}/{label}/{version}"
        shop.attachFile(attachment_uri, path)

def partshop_attach_genome_to_md(synbio_username, collection_name, file_path, email, password, chassis_label, version=1):
    shop = sbol2.PartShop("https://synbiohub.org")
    print(shop.login(email, password))

    label = "ModuleDefinition_" + chassis_label
    attachment_uri = f"https://synbiohub.org/user/{synbio_username}/{collection_name}/{label}/{version}"

    shop.attachFile(attachment_uri, file_path)

def create_synbio_collection(email, password, file_path, id, name, description, version='1'):
    response = requests.post(
        "https://synbiohub.org/login",
        headers={"Accept": "text/plain"},
        data={"email": email, "password": password}
    )
        
    if response.ok:
        token = response.text.strip() # theres a whitespace before the token for some reason
        response = requests.post(
        'https://synbiohub.org/submit',
        headers={
            'X-authorization': token,
            'Accept': 'text/plain'
        },
        files={
        'files': open(file_path,'rb'),
        },
        data={
            'id': id,
            'version' : version,
            'name' :  name,
            'description' : description,
            'citations' : '',
            'overwrite_merge' : '0'
        },
    
    )
    else:
        print("Login failed:", response.status_code)
        print(response.text)

def partshop_pull_collection(email, password, synbio_username, collection_name, file_path, version=1):
    shop = sbol2.PartShop("https://synbiohub.org")
    doc = sbol2.Document()
    shop.login(email, password)

    collection_uri = f"https://synbiohub.org/user/{synbio_username}/{collection_name}/{collection_name}_collection/{version}"
    s = shop.pull(collection_uri, doc)
    
    for obj in doc:
        print(obj)   
    
    doc.write(file_path)
    return doc

def partshop_pull_sample_designs(email, password, synbio_username, collection_name, sample_design_labels, output_dir, prefix, version=1):
    shop = sbol2.PartShop("https://synbiohub.org")
    shop.login(email, password)

    for sample_design_label in sample_design_labels: 
        doc = sbol2.Document()
     #f"https://synbiohub.org/user/cywong/Ecolipromoterexpdataalt/ModuleDefinition_sample_design_definition_0/1
        collection_uri = f"https://synbiohub.org/user/{synbio_username}/{collection_name}/{prefix}_{sample_design_label}/{version}"
        s = shop.pull(collection_uri, doc)
        doc.write(os.path.join(output_dir, sample_design_label + ".xml"))

def new_partshop_pull_sample_designs(email, password, synbio_username, collection_name, sample_design_labels, output_dir, prefix, version=1):

    shop = sbol2.PartShop("https://synbiohub.org")
    doc = sbol2.Document()
    shop.login(email, password)
    uris = []

    for sample_design_label in sample_design_labels: 
        uris.append(f"https://synbiohub.org/user/{synbio_username}/{collection_name}/{prefix}_{sample_design_label}/{version}")
        
    shop.pull(uris, doc)
    doc.write(os.path.join(output_dir, sample_design_label + ".xml"))
        
def partshop_submit(email, password, synbio_username, collection_name, file_path, version=1):
    shop = sbol2.PartShop("https://synbiohub.org")
    doc = sbol2.Document()
    shop.login(email, password)

    collection_uri = f"https://synbiohub.org/user/{synbio_username}/{collection_name}/{collection_name}_collection/{version}"
    s = shop.submit(collection_uri, doc)
    
    for obj in doc:
        print(obj)   
    
    doc.write(file_path)

def download_all_attachments(email, password, doc, file_path):
    shop = sbol2.PartShop("https://synbiohub.org")
    shop.login(email, password)
    for attachment in doc.attachments:
        shop.downloadAttachment(attachment.identity, filepath=file_path)

In [14]:
ecoli_genome_file_name = "E. coli.fasta"
env_email = os.getenv("SYNBIO_EMAIL")
env_password = os.getenv("SYNBIO_PASSWORD")

username = "cywong"
output_name = 'ecolipromoter_one.xml'
id = "Ecolipromoterexpdataalt"
name = "E coli promoter data exploration alternative"
description = "A collection containing the extracted E coli data from paper"
sbol_file_name = output_name
imported_sbol_file_name = "promoters_import.xml"

In [15]:
doc, chassis_label, sample_design_labels = create_sbol_attempt2_doc(df3, os.path.join(sbol_path, output_name))       

# # # #partshop_attach_exp_data(username, id, attachment_file_names, env_email, env_password, exp_labels)
# partshop_attach_genome_to_md(username, id, os.path.join(attachments_path, ecoli_genome_file_name), env_email, env_password, chassis_label)

In [8]:
create_synbio_collection(env_email, env_password, os.path.join(sbol_path, sbol_file_name), id, name, description)

In [7]:
partshop_pull_sample_designs(env_email, env_password, username, id, sample_design_labels, sbol_path, "ModuleDefinition")

In [ ]:
doc = partshop_pull_collection(env_email, env_password, username, id, os.path.join(downloaded_sbol_path, imported_sbol_file_name), version=1)

In [ ]:
doc2 = sbol2.Document()
doc2.addModuleDefinition(doc.get("https://synbiohub.org/user/cywong/Ecolipromoterexpdataalt/ModuleDefinition_sample_design_definition_0/1"))
doc2.write("dd.xml")

In [ ]:
sd = doc.get("https://synbiohub.org/user/cywong/Ecolipromoterexpdataalt/ModuleDefinition_sample_design_definition_0/1")
doc = sbol2.Document()
doc.addModuleDefinition(sd)
doc.write(os.path.join(sbol_path, "sample_design.xml"))

In [ ]:
# download_all_attachments(env_email, env_password, doc, pulled_attachments_path)

In [ ]:
def create_df_from_sbol_doc(doc, pulled_attachments_path):
    df_new = pd.DataFrame()
    for exp in doc.experimentalData:
        attachment = doc.get(exp.attachments[0])
        sample_design = doc.get(exp.wasDerivedFrom[0])
    
        # may need to be changed
        strain = doc.get(sample_design).modules[0] if 'strain' in doc.get(sample_design).modules[0].identity else doc.get(sample_design).modules[1]
        promoter =  doc.get(doc.get(strain.definition).functionalComponents[0].definition)
        promoter_seq = doc.get(promoter.sequences[0]).elements
        
        df1 = pd.read_csv(os.path.join(pulled_attachments_path, attachment.name))
        df1['Sequence'] = promoter_seq
        df_new = pd.concat([df_new, df1], ignore_index=True)

    return df_new


# for mod in doc.moduleDefinitions:
#     print("Mod: ", mod.identity)
# for attachment in doc.attachments:
#     print("Attachment: ", attachment.source) 
    
# for seq in doc.sequences:
#     print("Seq:", seq.elements) 

doc = sbol2.Document()
doc.read(os.path.join(downloaded_sbol_path, imported_sbol_file_name))

In [36]:
class Model(object):
    __metaclass__ = ABCMeta

    @abstractmethod
    def __init__(self, **hyperparameters):
        pass

    @abstractmethod
    def train(self, X, y, validation_data):
        pass

    @abstractmethod
    def predict(self, X):
        pass

    def test(self, X, y):
        return self.evaluate(X, y)
        # return ClassificationResult(y, self.predict(X))

    def score(self, X, y):
        pass

class DecisionTree(Model):

    def __init__(self):
        self.classifier = scikit_DecisionTree()

    def train(self, X, y, validation_data=None):
        self.classifier.fit(X, y)

    def predict(self, X):
        predictions = np.asarray(self.classifier.predict_proba(X))[..., 1]
        if len(predictions.shape) == 2:  # multitask
            predictions = predictions.T
        else:  # single-task
            predictions = np.expand_dims(predictions, 1)
        return predictions
    

class RandomForestRegression(DecisionTree):
    def __init__(self):
        self.regressor = RandomForestRegressor(n_estimators=100)

    def train(self, X, y, validation_data=None):
        # X shape: n_samples, n_features
        # y shape: n_samples
        self.regressor.fit(X, y)

    def predict(self, X):
        return self.regressor.predict(X)

    def score(self, X, y):
        predictions = np.squeeze(self.regressor.predict(X))
        return np.corrcoef(predictions, y)[0,1]

In [ ]:
train_df = pd.read_csv(os.path.join(model_data_path, "tss_expression_model_format_train_genome_split.txt"), sep='\t', header=None, names=["Sequence", "Expression"])
test_df = pd.read_csv(os.path.join(model_data_path, "tss_expression_model_format_test_genome_split.txt"), sep='\t',  header=None, names=["Sequence", "Expression"])

X_train, y_train = process_seqs(train_df, 150, "Sequence", "Expression")
X_test, y_test = process_seqs(test_df, 150, "Sequence", "Expression")

In [ ]:
def train_RandomForest(X_train, X_test, y_train, y_test):
    print("Running random forest regression...")
    model = RandomForestRegression()
    model.train(X_train, y_train)
    predictions = model.predict(X_test)
    
    with open("outputs.txt", 'w') as outfile:
        for i in range(len(predictions)):
            outfile.write(str(float(predictions[i])) + '\t' +
                      str(float(y_test[i])) + '\n')
    
    score = model.score(X_test, y_test)
    print("Score:", score)
    return model, predictions

In [ ]:
model, predictions = train_RandomForest(X_train, X_test, y_train, y_test)

In [ ]:
mse = mean_squared_error(y_test, predictions)
r_squared = r2_score(y_test, predictions)
rmse = np.sqrt(mse)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Square Error (RMSE): {rmse}")
print(f"R^2: {r_squared}")

In [ ]:
df = df.iloc[0:5000].sample(frac=1, random_state=42).reset_index(drop=True)
df['strand'] = df['strand'].map({'+': 0, '-': 1})
df.drop(['start', 'end'], axis=1, inplace=True)


columns_to_scale = ['RNA_exp_1', 'DNA_sum_1', 'DNA_sum_2', 'DNA_ave', 'variation']
scaler = StandardScaler()
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

In [19]:
X_sequences = df3['variant']
y = df3['expn_med'].values.reshape(-1, 1)

In [15]:
X_sequences_ohe = []
for i in range(len(X_sequences)):
    ohe = one_hot_encode_modern(X_sequences.iloc[i])
    X_sequences_ohe.append(ohe)

In [26]:
len(X_sequences_ohe)
y_scaled = StandardScaler().fit_transform(y)
y_scaled = y_scaled.ravel()


In [ ]:
a = []
for i in range(len(df)):
    seq = pad_sequence(df.loc[i, 'fragment'], 200)
    ohe = one_hot_encode_modern(seq)
    a.append(ohe)

In [ ]:
X = np.array(a)
X1 = X.reshape(X.shape[0], -1)
X2 = df.drop(['RNA_exp_ave', 'fragment'], axis=1).values
X = np.concatenate([X1, X2], axis=1)
y = np.log1p(df['RNA_exp_ave'])

In [ ]:
 df['RNA_exp_ave'].describe()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)


In [ ]:
X_train

In [ ]:
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [3, 7],
    'learning_rate': [0.01, 0.1],
    'colsample_bytree': [0.8, 1.0]
}

grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  
    cv=3,
    verbose=1,
)

grid = grid_search.fit(X_train, y_train)


In [ ]:
print("Best parameters:", grid_search.best_params_)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r_sq = r2_score(y_test, y_pred)
print("Test MSE:", mse)
print("R sq:", r_sq)

In [ ]:
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, colsample_bytree=1, learning_rate=0.1, max_depth=3, n_estimators=100)

In [ ]:
xgb_model.fit(X_train, y_train)

preds = xgb_model.predict(X_test)

r_sq = r2_score(y_test, preds) 

In [ ]:
r_sq

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X_sequences_ohe, y_scaled, test_size=0.2, random_state=42)

In [73]:
X_train = np.array(X_train) 
X_test = np.array(X_test)

In [74]:
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)


In [75]:
y_train = y_train.ravel()
y_test = y_test.ravel()

In [59]:
y_train

array([-0.09600204,  0.05270001, -0.07746396, ..., -0.10162205,
       -0.09888044, -0.10658665])

In [76]:
rfc = RandomForestRegression()
rfc.train(X_train, y_train)


In [77]:
preds = rfc.predict(X_test)


In [ ]:
r_sq = r2_score(y_test, preds) 


In [83]:
mean_squared_error(y_test, preds)


0.7960463681705944

In [82]:
r2_score(y_test, preds) 


0.20183621111801853